In [4]:
!pip install transformers langchain PyPDF2 accelerate

In [5]:
pip install langchain_community

In [6]:
pip install tiktoken

In [7]:
pip install torch torchvision torchaudio

In [8]:
pip install transformers_stream_generator

In [9]:
pip install -U langchain-community

In [10]:
pip install pypdf

In [11]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [12]:
pip install faiss-cpu

In [13]:
pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [14]:
pip install transformers langchain

In [ ]:
pip install --upgrade langchain langchain_core transformers

In [15]:
# 初始化分词器和模型（自动选择设备）
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    trust_remote_code=True,
    device_map="auto",  # 自动分配GPU/CPU
    torch_dtype=torch.float16  # 半精度节省显存
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NameError: name 'torch' is not defined

In [ ]:
import PyPDF2
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
# 1. PDF文本提取
loader = PyPDFLoader("2024-12-12-ENG-version-HSUHK-Introduction.pdf")
documents = loader.load()

# 2. 文本分块（关键参数优化）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,        # 每块512字符
    chunk_overlap=128,     # 重叠128字符保持上下文
    length_function=len
)
texts = text_splitter.split_documents(documents)

# 3. 向量化存储（使用bge-zh嵌入模型）
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# 1. 创建向量数据库
vector_db = FAISS.from_documents(texts, embeddings)

# 2. 定制Qwen提示模板
system_prompt = """你是一个专业的HSUHK问答助手，回答需基于以下上下文：
{context}
请用中文给出准确答案，若问题超出知识库范围请说明。"""

# 3. 构建RAG流水线
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",       # 简单全量检索模式
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={
        "prompt_template": system_prompt,
        "return_source_documents": True
    }
)

# 4. 问答接口
def ask_hsuhk(question):
    result = qa_chain.invoke({"query": question})
    return result["result"] if result["result"] else "未检测到相关信息"

In [16]:
while True:
    user_input = input("请输入问题（输入'退出'结束）：")
    if user_input.lower() == '退出':
        break
    print("回答：", ask_hsuhk(user_input))

KeyboardInterrupt: Interrupted by user